# Long-Term Procedural Memory with LangGraph

This notebook demonstrates how to implement long-term procedural memory using embeddings and vector search.

## Key Concepts:
- **Procedural memory**: Stores skills, workflows, processes, and best practices
- **Procedures**: Step-by-step instructions on how to accomplish tasks
- **Skill accumulation**: Agent learns and remembers techniques from experience
- **Process refinement**: Stores successful methods and patterns
- **Contextual application**: Retrieve and apply relevant procedures when needed

In [8]:
# Install required dependencies
import subprocess
import sys

packages = [
    "langgraph",
    "langchain",
    "langchain-openai",
    "langchain-chroma",
    "python-dotenv"
]

for package in packages:
    try:
        __import__(package.replace("-", "_"))
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])

print("✓ All dependencies installed")

✓ All dependencies installed


In [9]:
import os
from datetime import datetime
from dotenv import load_dotenv
from typing import Annotated
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from typing_extensions import TypedDict

# Verify API key
api_key = " "
if not api_key:
    print("⚠️  Warning: OPENAI_API_KEY not found. Please set it in your .env file")
else:
    print("✓ OpenAI API key loaded")

✓ OpenAI API key loaded


## Step 1: Initialize Embeddings and Vector Store

Setup the procedural memory system with embeddings and a vector database for storing procedures.

In [10]:
try:
    # Initialize embeddings
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=api_key)
    
    # Initialize vector store for procedures
    procedure_store = Chroma(
        collection_name="procedural_memory",
        embedding_function=embeddings,
    )
    
    print("✓ Embeddings and vector store initialized")
    
except Exception as e:
    print(f"✗ Error initializing embeddings: {e}")
    print("Make sure OPENAI_API_KEY is set")

✓ Embeddings and vector store initialized


## Step 2: Define the Agent State

The state includes:
- `messages`: Current conversation
- `procedures`: Skills and workflows learned

In [11]:
def add_messages(left: list[BaseMessage], right: list[BaseMessage]) -> list[BaseMessage]:
    """Append messages to existing messages."""
    return left + right

def add_procedures(left: list[str], right: list[str]) -> list[str]:
    """Append procedures to existing procedures."""
    return left + right

class AgentState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]
    procedures: Annotated[list[str], add_procedures]

print("✓ Agent state defined with short-term and procedural memory")

✓ Agent state defined with short-term and procedural memory


## Step 3: Create Agent Nodes

1. **extract_procedure**: Extract workflows and skills from conversations
2. **retrieve_procedures**: Find relevant learned techniques
3. **agent**: Respond with procedural guidance

In [12]:
try:
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, openai_api_key=api_key)
    
    def is_valid_procedure(text: str) -> bool:
        """
        Check if extracted text is a real procedure, not meta-commentary.
        """
        # Filter out meta-responses
        meta_phrases = [
            "no procedure",
            "no step",
            "it appears that",
            "does not contain",
            "if there were",
            "if you have",
            "please provide",
            "cannot create",
        ]
        
        lower_text = text.lower()
        if any(phrase in lower_text for phrase in meta_phrases):
            return False
        
        # Must have procedure/workflow indicators
        procedure_indicators = ["step", "process", "method", "procedure", "approach", "workflow", "1.", "first", "then", "next"]
        return any(indicator in lower_text for indicator in procedure_indicators)
    
    def get_procedure_summary(text: str) -> str:
        """
        Extract a clean title from procedure text.
        """
        lines = text.split('\n')
        for line in lines:
            if line.strip() and len(line.strip()) > 10:
                # Return first meaningful line as title
                return line.strip()[:80]
        return "Untitled Procedure"
    
    def extract_procedure(state: AgentState) -> dict:
        """
        Extract workflows, skills, or procedures from the latest user message.
        Store them in the vector store for future reference.
        """
        messages = state["messages"]
        
        # Get the last user message
        last_user_msg = None
        for msg in reversed(messages):
            if isinstance(msg, HumanMessage):
                last_user_msg = msg.content
                break
        
        if not last_user_msg:
            return {}
        
        print("🔧 Extracting procedures/skills from message...")
        
        # Use LLM to extract procedures
        extraction_prompt = f"""Extract ONLY one main procedure, workflow, or skill from this message if it describes how to do something.

Message: {last_user_msg}

For the procedure, include:
- What the procedure/skill is (name and purpose)
- Step-by-step instructions (numbered or clear steps)
- When to use it (context and applicable scenarios)
- Benefits or outcomes

Format as a clear workflow guide.
If there is NO procedure or how-to information mentioned, respond with: "No procedures to extract."
"""
        
        procedure_response = llm.invoke(extraction_prompt)
        procedure_text = procedure_response.content
        
        # Store procedure in vector store
        if procedure_text.strip() and "no procedure" not in procedure_text.lower():
            # Split by double newline but filter carefully
            procedures = [p.strip() for p in procedure_text.split("\n\n") if p.strip() and len(p.strip()) > 30]
            
            # Filter out meta-responses
            procedures = [proc for proc in procedures if is_valid_procedure(proc)]
            
            if procedures:
                # Add timestamp to procedures for storage
                timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                timestamped_procedures = [
                    f"[Learned on: {timestamp}]\n{proc}" 
                    for proc in procedures
                ]
                
                print(f"💾 Storing {len(procedures)} procedure(s):\n")
                for i, proc in enumerate(procedures, 1):
                    summary = get_procedure_summary(proc)
                    print(f"   Procedure {i}: {summary}")
                print()
                
                procedure_store.add_texts(timestamped_procedures)
                return {"procedures": procedures}
        
        print("⊘ No procedures extracted from this message")
        return {}
    
    def retrieve_procedures(state: AgentState) -> dict:
        """
        Retrieve relevant procedures from procedural memory based on current task.
        """
        messages = state["messages"]
        
        # Get the last message
        last_msg = messages[-1].content if messages else ""
        
        if not last_msg or len(procedure_store.get()["ids"]) == 0:
            return {}
        
        print("🔍 Retrieving relevant procedures from memory...")
        
        # Search for relevant procedures
        results = procedure_store.similarity_search(last_msg, k=3)
        
        if results:
            # Filter out meta-commentary from retrieved results
            valid_results = [doc for doc in results if is_valid_procedure(doc.page_content)]
            
            if valid_results:
                print(f"✓ Found {len(valid_results)} relevant procedure(s):\n")
                for i, doc in enumerate(valid_results, 1):
                    summary = get_procedure_summary(doc.page_content)
                    print(f"   Retrieved Procedure {i}: {summary}")
                print()
                retrieved_procedures = "\n".join([doc.page_content for doc in valid_results])
                return {"procedures": [f"[Known procedures]\n{retrieved_procedures}"]}
            else:
                print("⊘ No relevant procedures found")
                return {}
        
        print("⊘ No relevant procedures found")
        return {}
    
    def agent(state: AgentState) -> dict:
        """
        Agent that processes requests with procedural memory context.
        """
        messages = state["messages"]
        procedures = state.get("procedures", [])
        
        # Build context from procedures
        context = ""
        if procedures:
            context = f"\nRelevant procedures and skills you know:\n" + "\n".join(procedures)
        
        # Prepare system message with procedure context
        system_msg = f"""You are a helpful assistant with access to long-term procedural memory.
You remember workflows, skills, and best practices you've learned.
        
{context}

Use the known procedures and skills to provide practical guidance and apply learned techniques."""
        
        # Call LLM with full context
        response = llm.invoke([
            {"role": "system", "content": system_msg},
            *messages
        ])
        
        return {"messages": [response]}
    
    print("✓ Agent nodes created successfully")

except Exception as e:
    print(f"✗ Error creating agent nodes: {e}")
    import traceback
    traceback.print_exc()

✓ Agent nodes created successfully


## Step 4: Build the Graph

The workflow:
1. Extract procedures/skills from user messages
2. Retrieve relevant learned techniques
3. Agent responds with procedural guidance

In [13]:
# Create the graph
graph_builder = StateGraph(AgentState)

# Add nodes
graph_builder.add_node("extract_procedure", extract_procedure)
graph_builder.add_node("retrieve_procedures", retrieve_procedures)
graph_builder.add_node("agent", agent)

# Add edges
graph_builder.add_edge(START, "extract_procedure")
graph_builder.add_edge("extract_procedure", "retrieve_procedures")
graph_builder.add_edge("retrieve_procedures", "agent")
graph_builder.add_edge("agent", END)

# Compile the graph
graph = graph_builder.compile()

print("✓ Graph compiled successfully")

✓ Graph compiled successfully


## Step 5: Run a Multi-Turn Conversation Demo

Demonstrate how procedural memory learns and applies techniques.

In [14]:
try:
    # Clear the procedure store for a fresh demo
    print("🗑️  Clearing previous procedures for a fresh start...")
    if len(procedure_store.get()["ids"]) > 0:
        procedure_store.delete(procedure_store.get()["ids"])
    print("✓ Store is empty\n")
    
    # Initialize conversation with procedural memory
    initial_state = {
        "messages": [],
        "procedures": []
    }
    
    # Multi-turn conversation learning procedures
    conversation_turns = [
        "I've developed a workflow for debugging Python code. First, I enable detailed logging to see execution flow. Second, I use a debugger to set breakpoints at suspected error locations. Third, I inspect variable values at each breakpoint. Finally, I isolate the problematic code section and test fixes incrementally.",
        "What debugging techniques do I know?",
        "I'm running into a mysterious error in my code. Can you help?",
        "How would you approach finding this bug systematically?"
    ]
    
    print("=" * 60)
    print("LONG-TERM PROCEDURAL MEMORY DEMONSTRATION")
    print("=" * 60)
    print()
    
    current_state = initial_state.copy()
    
    for i, user_input in enumerate(conversation_turns, 1):
        print(f"Turn {i}: User Input")
        print(f">> {user_input}")
        print()
        
        # Add user message to state
        current_state["messages"].append(HumanMessage(content=user_input))
        
        # Run the agent
        result = graph.invoke(current_state)
        current_state = result
        
        # Extract and display agent response
        last_message = current_state["messages"][-1]
        print(f"Turn {i}: Agent Response")
        print(f"<< {last_message.content}")
        print()
        
        # Show procedural memory size
        stored_count = len(procedure_store.get()["ids"])
        print(f"📚 Procedures stored in memory: {stored_count}")
        print("-" * 60)
        print()
    
    print("✓ Multi-turn conversation completed")
    print(f"✓ Total procedures stored in procedural memory: {len(procedure_store.get()['ids'])}")

except Exception as e:
    print(f"✗ Error running conversation: {e}")
    import traceback
    traceback.print_exc()

🗑️  Clearing previous procedures for a fresh start...
✓ Store is empty

LONG-TERM PROCEDURAL MEMORY DEMONSTRATION

Turn 1: User Input
>> I've developed a workflow for debugging Python code. First, I enable detailed logging to see execution flow. Second, I use a debugger to set breakpoints at suspected error locations. Third, I inspect variable values at each breakpoint. Finally, I isolate the problematic code section and test fixes incrementally.

🔧 Extracting procedures/skills from message...
💾 Storing 5 procedure(s):

   Procedure 1: **Workflow for Debugging Python Code**
   Procedure 2: **Purpose:** To systematically identify and fix errors in Python code using a st
   Procedure 3: **Step-by-Step Instructions:**
   Procedure 4: **When to Use It:**
   Procedure 5: **Benefits or Outcomes:**

🔍 Retrieving relevant procedures from memory...
✓ Found 3 relevant procedure(s):

   Retrieved Procedure 1: [Learned on: 2025-12-11 22:13:04]
   Retrieved Procedure 2: [Learned on: 2025-12-11 22:1

## Key Takeaways: Long-Term Procedural Memory Pattern

### How it Works:
1. **Extract**: Workflows, skills, and procedures are extracted from user messages
2. **Document**: Procedures are stored with timestamps showing when learned
3. **Store**: Step-by-step instructions are stored in the vector database
4. **Retrieve**: Relevant techniques are retrieved based on task context
5. **Apply**: Agent uses learned procedures to provide practical guidance

### Characteristics:
- ✅ Remembers how to do things (not just facts or events)
- ✅ Stores step-by-step instructions and workflows
- ✅ Improves over time as more procedures are learned
- ✅ Can apply multiple techniques to solve problems
- ✅ Helps agents become more capable and efficient
- ❌ Requires explicit procedural descriptions
- ❌ Quality depends on how well procedures are taught

### Differences from Other Memory Types:

| Aspect | Semantic | Episodic | Procedural |
|--------|----------|----------|-----------|
| **Content** | Facts & knowledge | Events & experiences | Skills & workflows |
| **Temporal** | Timeless | Time-aware | Learning progression |
| **Context** | What is known | What happened | How to do things |
| **Use case** | General knowledge | Personal history | Task execution |
| **Example** | "Python is a language" | "I attended a conference" | "How to debug code" |

### Common Use Cases:
- Code generation with learned patterns and techniques
- Customer support with documented solutions
- Workflow automation learning best practices
- Educational systems building skill progressions
- Software development remembering coding patterns